This code is © P. Sánchez and I. González, 2025, and it is made available under the GPL license enclosed with the software.

If you use this software in your research or projects, we kindly ask you to acknowledge our work by citing the following reference:

- Sánchez, P., González, I., Cortés, A., Carrillo, C., & Margalef, T. (2025). *Airborne Hyperspectral Data Processing* [Software]. Zenodo. [https://doi.org/10.5281/zenodo.14871861](https://doi.org/10.5281/zenodo.14871861)

This code was developed as part of the **SALUS (CPP2021-008762)** framework.

# Airborne Data (II)

## Automatic Extraction of Wildfire Perimeters  

This code is designed to automatically extract wildfire perimeters by detecting hotspots from airborne data. While the methodology is automated, certain parameters need to be adjusted depending on various factors, such as the type and quality of the data, fire intensity, smoke presence, and other environmental conditions.  

The objective of this process is to generate a fire perimeter similar to the one that would be manually obtained using QGIS, but now using Python. The methodology relies on two key indices:  

- **Burned Area Index (BAI)**  
- **Long-Wave Infrared (LWIR) Imagery**  

By leveraging these indices, we can achieve a faster processing workflow. However, it is important to note that the resulting burned area may be less precise compared to manual processing.  

### Methodology for Burned Area Extraction  

The automated processing of burned areas follows a three-step methodology, fully implemented in Python:  

1. **High percentiles to identify hotspots (LWIR) and burned areas (BAI)** – Hotspots and burned areas are detected by selecting pixels with values above a high percentile threshold in the LWIR and BAI images, respectively.  
2. **Filter small hotspots and burned areas** – Small hotspots and small burned areas are discarded based on a threshold relative to the largest detected hotspot or burned area to reduce noise and false positives.  
3. **Create a concave hull** – The remaining hotspots and burned areas are used to generate a concave hull, forming the final fire perimeter.  

These steps will be explained in more detail throughout the code.  

### Case Study: Wildfire in Fundão, Portugal  

For the practical example, we have selected a wildfire that occurred in Fundão, Portugal. Airborne data is available for three distinct time periods, allowing us to analyze the fire's progression over time. The three time instances we will process correspond to:  

- **3:00 p.m.**  
- **4:30 p.m.**  
- **6:00 p.m.**  

### Initial Data Visualization  

Before processing the data, we will first visualize the **RGBN** and **thermal (LWIR)** images for these time periods in QGIS. This step helps us understand how the fire evolved and assess the quality of the available data. Based on the images, we can make the following observations:  

| **Time**   | **LWIR Observations**  | **RGB Observations**   |  
|------------|------------------------|------------------------|  
| **3:00 p.m.**  | Active fire zone       | Dispersed smoke        |  
| **4:30 p.m.**  | Moderate fire zone     | Dispersed smoke        |  
| **6:00 p.m.**  | Residual heat          | Concentrated smoke     |  

These observations will guide us in fine-tuning the parameters and ensuring the most accurate extraction of the fire perimeter. In the next steps, we will proceed with processing the airborne data to automatically extract the burned area.  


## Implementation in Python  

The following Python code implements the methodology described above to automatically extract the wildfire perimeter. Run the code step by step, adjusting parameters if needed to optimize the results for different fires.  


If any of the libraries do not work, you can install them using the following command:

pip install `<library_name>`

Replace `<library_name>` with the name of the library you need to install.


In [1]:
import os
import rasterio
from rasterio.features import shapes
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import unary_union
from shapely.geometry import MultiPoint
import fiona
from shapely import concave_hull

The folder path must be changed depending on the time period being processed. All generated outputs will be saved in the same folder you specify here. The possible folders are: **20240913150443, 20240913163416, 20240913175528**. The following table shows the correspondence between the time and the folder name:


  | Time       | Folder |
|------------|--------------------|
| 3:00 p.m.  | 20240913150443     |
| 4:30 p.m.  | 20240913163416     |
| 6:00 p.m.  | 20240913175528     |

In [2]:
# Define the specific folder
folder_path = "20240913175528"
folder = os.path.basename(folder_path)

## Extracting Raw Hotspots from LWIR Data

In this section, we focus solely on extracting the raw hotspots from the **LWIR (Long-Wave Infrared)** data. This is done by applying a threshold based on a chosen percentile to identify the most intense heat pixels, which represent potential hotspots.

### Key Parameter:
- **Threshold Percentile (`threshold_percentile`)**: The percentile value is used to set the threshold above which the pixels are considered part of a hotspot. A common starting point is **98%**, but this can be adjusted depending on the data quality and fire intensity.

### Adapting the Threshold Percentile:
The value of `threshold_percentile` determines how sensitive the hotspot extraction process is to changes in temperature. Here's how to adapt the threshold:

1. **High Percentile (e.g., 98-99%)**:
   - **Effect**: Fewer, but more intense hotspots are detected. This is useful if you're interested in isolating the most significant fire activity, such as large or intense fire sources.
   - **When to Use**: This is ideal when the fire is very intense, and you're looking for the highest heat values in the scene.

2. **Lower Percentile (e.g., 95-98%)**:
   - **Effect**: More, but less intense hotspots are detected, even those that are not burning strongly.
   - **When to Use**: This is useful for identifying the full extent of a fire, including areas of lower intensity.

3. **Dynamic Adjustment**:
   - It's often helpful to visually inspect the hotspots after applying different threshold values. If the hotspots are too sparse and you're interested in the main fire areas, try raising the threshold percentile. If you're missing smaller heat sources, lower the percentile.

By adjusting the `threshold_percentile`, you can fine-tune the hotspot extraction to suit the specific characteristics of the data and the fire's behavior.

### Methodology for Raw Hotspot Extraction:
1. **Extract Hotspots**: Using the `threshold_percentile`, we identify the pixels in the LWIR image that exceed this threshold, representing the hottest areas.
2. **Save the Raw Hotspot Data**: We save the raw hotspot data without any filtering based on size. These hotspots are saved as geometries in a `.shp` file for further analysis.

In [3]:
LWIR_tiff = os.path.join(folder_path,"LWIR_normalized.tiff")

# Threshold for detection (high percentiles to highlight fires)
threshold_percentile = 98

if os.path.exists(LWIR_tiff):  # Check if the LWIR file exists
    with rasterio.open(LWIR_tiff) as src:
        transform = src.transform
        crs = src.crs
        band = src.read(1)

        threshold = np.percentile(band, threshold_percentile)
        binary = (band >= threshold).astype(np.uint8)

        # Extract geometries
        geometries = [shape(geom) for geom, value in shapes(binary, transform=transform) if value == 1]

        if geometries:
            # Save the unprocessed hotspots
            output_filename_raw_LWIR = os.path.join(folder_path,"hotspot_LWIR_raw.shp")

            # Create schema and save the raw hotspots
            schema = {'geometry': 'Polygon', 'properties': {'area': 'float'}}
            with fiona.open(output_filename_raw_LWIR, 'w', driver='ESRI Shapefile', crs=crs, schema=schema) as shp:
                for geom in geometries:
                    shp.write({'geometry': mapping(geom), 'properties': {'area': geom.area}})

            print(f"Raw LWIR hotspots saved to {output_filename_raw_LWIR}")
        else:
            print(f"No hotspots detected using LWIR.")


Raw LWIR hotspots saved to 20240913175528/hotspot_LWIR_raw.shp


## Filtering LWIR Hotspots by Area

After the hotspots are identified, the next step is to filter them based on their area. This filtering is essential because not all detected hotspots are relevant; some may be noise, especially in scenarios where smoke affects the data. To filter irrelevant hotspots, we calculate the area of each hotspot and apply a threshold. Only hotspots that have an area greater than or equal to the threshold are retained.


### How the Threshold Works

The area threshold is calculated as a percentage of the area of the largest detected hotspot. For example, if the largest hotspot in the data has an area of 1000 pixels, and we set the threshold to 1%, the minimum acceptable area for a hotspot will be 10 pixels. Hotspots smaller than this will be discarded.

This threshold is adjustable and must be defined by the user based on the quality of the raw hotspots:

- **If there is a lot of smoke**: Smoke can cause isolated, small hotspots that are likely noise. In such cases, the user should set a higher threshold (like `0.5`) to remove these small, isolated hotspots. This ensures that only larger, more likely fire hotspots are kept.
  
- **If the hotspots are clustered**: In situations where the hotspots are well-clustered and there is little smoke interference, a lower threshold (or even zero) can be used, as the small hotspots in the cluster are likely part of the same fire. In this case, there is no need to remove small hotspots.

The `threshold_area_LWIR` is a parameter that needs to be adjusted based on how the raw hotspots appear in your data. If the raw hotspots are highly dispersed due to smoke, increase the threshold. If they are more tightly clustered, decrease the threshold, or set it to zero to retain even the smallest hotspots.

We recommend using this `threshold_area_LWIR` parameter for the practical example. If you have time, you can experiment with these parameters to obtain better results for the fire perimeter.  

| Time       | threshold_area_LWIR |
|------------|---------------------|
| 3:00 p.m.  | 0.01                |
| 4:30 p.m.  | 0.01                |
| 6:00 p.m.  | 0.01                |

## Error Handling

If, after applying the area threshold, all hotspots are discarded because their area was too small, you may encounter an error message like:

> "All fires detected using LWIR are too small"

This indicates that the `threshold_area_LWIR` is too high, and you will need to lower it to retain valid hotspots.

In [4]:
# Area threshold adjustment
threshold_area_LWIR = 0.01 #Change accordingly

hotspot_lwir = []

# Open the raw LWIR shapefile
with fiona.open(output_filename_raw_LWIR, 'r') as src:
    crs = src.crs
    geometries = [shape(feature['geometry']) for feature in src]

    # Calculate areas of the geometries
    areas = [geom.area for geom in geometries]

    if not areas:
        print(f"No fires detected using LWIR.")
    else:
        # Calculate the area threshold based on the largest geometry
        max_area = max(areas)
        area_threshold = max_area * threshold_area_LWIR

        # Filter geometries based on the area threshold
        filtered_geometries = [geom for geom in geometries if geom.area >= area_threshold]

        if not filtered_geometries:
            print(f"All fires detected using LWIR are too small.")
        else:
            # Merge filtered geometries into a single geometry
            merged_geometry = unary_union(filtered_geometries)
            hotspot_lwir.append(merged_geometry)

            # Define the output file path for the filtered hotspots
            output_filename_LWIR =os.path.join(folder_path, "hotspot_LWIR.shp")

            # Save filtered hotspots to a new shapefile
            if hotspot_lwir:
                schema = {'geometry': 'Polygon', 'properties': {'area': 'float'}}
                combined_LWIR_geometry = unary_union(hotspot_lwir)
                with fiona.open(output_filename_LWIR, 'w', driver='ESRI Shapefile', crs=crs, schema=schema) as shp:
                    if combined_LWIR_geometry.geom_type == 'Polygon':
                        shp.write({'geometry': mapping(combined_LWIR_geometry), 'properties': {'area': combined_LWIR_geometry.area}})
                    elif combined_LWIR_geometry.geom_type == 'MultiPolygon':
                        for geom in combined_LWIR_geometry.geoms:
                            shp.write({'geometry': mapping(geom), 'properties': {'area': geom.area}})

                print(f"LWIR hotspots (filtered) saved to {output_filename_LWIR}")

LWIR hotspots (filtered) saved to 20240913175528/hotspot_LWIR.shp


## Extracting Raw Burned Areas from BAI Data

The extraction process for **BAI (Burned Area Index)** data differs from LWIR in that it focuses on detecting **burned areas** rather than **hotspots**. The threshold used in BAI is based on **fire extension** (i.e., the spread of the burn) rather than **fire intensity**.

### Key Considerations for BAI Data:

1. **Focus on Burned Area Extension**: Unlike LWIR, which identifies heat sources or hotspots related to active fire, **BAI identifies the extent of burned areas**. This means the threshold for BAI extraction does not depend on the intensity of the fire but on the spread of the fire, or how much area has already been burned.

2. **Threshold Percentile for BAI**: The **threshold_percentile_BAI** is used to define the extent of the burned area based on the changes in reflectance.
   - Higher percentiles (e.g., 98-99%) capture the most severely burned regions.
   - Lower percentiles (e.g., 95%) allow for the detection of larger burned areas, including those that are only partially burned.

### Why This Matters:

Since **BAI focuses on burned area extension**, the threshold is used to define the **area** affected by the fire. This means the threshold value should be adjusted based on the extension of the already burned area rather than fire intensity.

In [5]:
BAI_tiff = os.path.join(folder_path,"BAI_index.tiff")

# Threshold for detection
threshold_percentile = 98

if os.path.exists(BAI_tiff):  # Check if the BAI file exists
    with rasterio.open(BAI_tiff) as src:
        transform = src.transform
        crs = src.crs
        band = src.read(1)

        threshold = np.percentile(band, threshold_percentile)
        binary = (band >= threshold).astype(np.uint8)

        # Extract geometries
        geometries = [shape(geom) for geom, value in shapes(binary, transform=transform) if value == 1]

        if geometries:
            # Save the unprocessed burned areas
            output_filename_raw_BAI = os.path.join(folder_path,"burnedarea_BAI_raw.shp")

            # Create schema and save the raw burned areas
            schema = {'geometry': 'Polygon', 'properties': {'area': 'float'}}
            with fiona.open(output_filename_raw_BAI, 'w', driver='ESRI Shapefile', crs=crs, schema=schema) as shp:
                for geom in geometries:
                    shp.write({'geometry': mapping(geom), 'properties': {'area': geom.area}})

            print(f"Raw BAI burned areas saved to {output_filename_raw_BAI}")
        else:
            print(f"No burned areas detected using BAI.")


Raw BAI burned areas saved to 20240913175528/burnedarea_BAI_raw.shp


## Filtering Burned Areas from BAI Data

After extracting the raw burned areas from **BAI (Burned Area Index)** data, it is important to filter out small, irrelevant areas that may be caused by **smoke interference**. Smoke can lead to false positives, resulting in the detection of small burned areas that are not representative of the fire's extent.

### How the Filtering Threshold Works:

The threshold for filtering is set based on the **size of the burned areas** and is defined by a percentage of the largest detected burned area. Burned areas smaller than this threshold are discarded as they are often noise caused by smoke or other environmental factors.

1. **Higher Threshold for Smoky Areas**: In regions with significant smoke, BAI data may contain small, isolated burned areas caused by smoke, not actual fire. In such cases, it is advisable to set a higher threshold (`0.5`) to filter out these small, scattered burned areas. A higher threshold will help ensure that only larger, more contiguous burned areas are retained, reducing the impact of smoke on the data.

2. **Lower Threshold for Cleaner Data**: In areas where smoke interference is minimal and the burned areas are well-clustered, a lower threshold (`0.01`) can be used. This allows for capturing more details of the fire's spread, including smaller burned areas that are still relevant.

### Why This Matters More using BAI than LWIR:

Because BAI is **much more sensitive to smoke**, it may detect small, irrelevant burned areas that are not indicative of the actual fire's extent. Filtering helps remove these false positives, allowing for a clearer representation of the fire's true extent. The **threshold for filtering** should therefore be adjusted based on the level of smoke interference. If there is significant smoke, a higher threshold is necessary to remove small, isolated burned areas caused by smoke. Conversely, if the fire has minimal smoke, a lower threshold can be applied.

We recommend using this `threshold_area_BAI` parameter for the practical example. If you have time, you can experiment with other values for the parameter to obtain better results for the fire perimeter.  

| Time       | threshold_area_BAI |
|------------|--------------------|
| 3:00 p.m.  | 0.5                |
| 4:30 p.m.  | 0.5                |
| 6:00 p.m.  | 0.01               |


In [6]:
# Area threshold adjustment
threshold_area_BAI = 0.5 #Change accordingly

burnedarea_BAI = []

# Open the raw BAI shapefile
with fiona.open(output_filename_raw_BAI, 'r') as src:
    crs = src.crs
    geometries = [shape(feature['geometry']) for feature in src]

    # Calculate areas of the geometries
    areas = [geom.area for geom in geometries]

    if not areas:
        print(f"No burned areas detected using BAI.")
    else:
        # Calculate the area threshold based on the largest geometry
        max_area = max(areas)
        area_threshold = max_area * threshold_area_BAI

        # Filter geometries based on the area threshold
        filtered_geometries = [geom for geom in geometries if geom.area >= area_threshold]

        if not filtered_geometries:
            print(f"All burned areas detected using BAI are too small.")
        else:
            # Merge filtered geometries into a single geometry
            merged_geometry = unary_union(filtered_geometries)
            burnedarea_BAI.append(merged_geometry)

            # Define the output file path for the filtered burned area
            output_filename_bai =  os.path.join(folder_path,"burnedarea_BAI.shp")

            # Save filtered burned areas to a new shapefile
            if burnedarea_BAI:
                schema = {'geometry': 'Polygon', 'properties': {'area': 'float'}}
                combined_bai_geometry = unary_union(burnedarea_BAI)
                with fiona.open(output_filename_bai, 'w', driver='ESRI Shapefile', crs=crs, schema=schema) as shp:
                    if combined_bai_geometry.geom_type == 'Polygon':
                        shp.write({'geometry': mapping(combined_bai_geometry), 'properties': {'area': combined_bai_geometry.area}})
                    elif combined_bai_geometry.geom_type == 'MultiPolygon':
                        for geom in combined_bai_geometry.geoms:
                            shp.write({'geometry': mapping(geom), 'properties': {'area': geom.area}})

                print(f"BAI burned areas (filtered) saved to {output_filename_bai}")

BAI burned areas (filtered) saved to 20240913175528/burnedarea_BAI.shp


## Combining BAI Burned Areas and LWIR Hotspots to Determine Fire Extension

To estimate the **current extension of a fire** at a specific timestamp, it's essential to combine both **BAI Burned areas** and **LWIR hotspots**. Each data provides different but complementary insights into the fire's behavior, allowing for a more accurate understanding of its spatial spread.

- **BAI Burned Areas**: The BAI index indicates areas that have already burned. It represents the **fire's footprint**, showing the extent of the area affected by fire. This is crucial for understanding the overall area of destruction, even if the fire is no longer actively burning.

- **LWIR Hotspots**: On the other hand, LWIR hotspots capture areas with high thermal values, often associated with ongoing fire activity. These hotspots can indicate **active fire** and help identify the regions where the fire is currently spreading or intensifying.

### Why Combine Both Datasets?

Combining these two datasets is important for creating a comprehensive view of the fire's **spatial extension**. By merging BAI burned areas with LWIR hotspots, we can achieve the following:

1. **Capturing Ongoing and Past Fire Activity**: BAI shows where fire has already occurred, while LWIR hotspots identify where the fire is still active. By combining both, we capture the **total fire extension**, both in terms of already burned areas and areas with ongoing heat sources. 

2. **More Accurate Fire Mapping**: Relying on only one of these datasets could lead to incomplete or misleading maps. For example, using only BAI might ignore the ongoing spread of the fire, while using only hotspots might miss areas that have already been burned but still have residual heat. The combination of both gives a clearer and more accurate picture.

3. **Improved Fire Management and Response**: For fire management, understanding the current and past extension of the fire is critical for resource allocation and decision-making.

In [7]:
if hotspot_lwir and burnedarea_BAI:
    all_fire = burnedarea_BAI + hotspot_lwir
    combined_geometry = unary_union(all_fire)

    output_filename_combined = os.path.join(folder_path,"combined_fire.shp")

    if combined_geometry and not combined_geometry.is_empty:
        schema = {'geometry': 'Polygon', 'properties': {'area': 'float'}}
        with fiona.open(output_filename_combined, 'w', driver='ESRI Shapefile', crs=crs, schema=schema) as shp:
            if combined_geometry.geom_type == 'Polygon':
                shp.write({'geometry': mapping(combined_geometry), 'properties': {'area': combined_geometry.area}})
            elif combined_geometry.geom_type == 'MultiPolygon':
                for geom in combined_geometry.geoms:
                    shp.write({'geometry': mapping(geom), 'properties': {'area': geom.area}})

        print(f"Combined shapefile saved to {output_filename_combined}")
    else:
        print(f"No significant fires information.")

Combined shapefile saved to 20240913175528/combined_fire.shp


## Create a Concave Hull for Fire Extension

Once the **BAI burned areas** and **LWIR hotspots** have been combined, the next step is to derive the **current fire extension** approximately. This is done by generating a **concave hull** around the points obtained from both datasets.

A **concave hull** is a polygon that tightly encloses a set of points.

### Ratio Parameter for Concave Hull

The concave hull requires a **ratio** parameter, which determines how tightly the shape fits around the points. A ratio of `0.3` is typically used, as it slightly overestimates the area, which is generally preferred in fire monitoring to ensure safety and a more cautious approach. However, you can adjust this ratio depending on the level of precision you need:
- **Lower ratio (e.g., 0.1)**: Results in a more detailed shape with tighter curves, fitting closely to the actual data points.
- **Higher ratio (e.g., 0.5)**: Produces a more relaxed shape that is less tightly fitted to the points, potentially overestimating the fire's boundary.

### Important Considerations:
- **Too Low of a Ratio**: If the ratio is too low, the concave hull may form multiple disconnected fragments instead of a continuous shape. This happens because the shape tries to fit each point very closely, which can lead to undesirable gaps.
  
- **Too High of a Ratio**: If the ratio is too high, the concave hull will become too inflated, covering a larger area than necessary. This could lead to overestimating the fire's true extent.

By adjusting the ratio appropriately, you can generate a more reliable and meaningful representation of the fire's current extension.

In [8]:
ratio = 0.3

# Generate the concave hull from BAI burned areas and LWIR hotspots
points = []
with fiona.open(output_filename_combined, 'r') as src:
    for feature in src:
        geom = shape(feature['geometry'])
        if geom.geom_type == 'Polygon':
            points.extend(geom.exterior.coords)
        elif geom.geom_type == 'MultiPolygon':
            for poly in geom:
                points.extend(poly.exterior.coords)

multipoint = MultiPoint(points)

concave_hull_geom = concave_hull(multipoint, ratio=ratio)  # Generate the concave hull

concave_output_filename = os.path.join(folder_path,"concave_combined.shp")

schema = {'geometry': 'Polygon', 'properties': {}}
with fiona.open(output_filename_combined, 'r') as src:
    with fiona.open(concave_output_filename, 'w', driver='ESRI Shapefile', crs=src.crs, schema=schema) as dst:
        dst.write({'geometry': mapping(concave_hull_geom), 'properties': {}})

print(f"Concave hull saved to {concave_output_filename}")

Concave hull saved to 20240913175528/concave_combined.shp


## Conclusion

To accurately estimate the extension of a fire, it is essential to combine multiple data sources, such as BAI burned areas and LWIR hotspots, along with other relevant products. Each of these datasets provides different insights into the fire's behavior, and their integration allows for a more comprehensive and precise assessment of the fire's current and past extent. Relying on a single source could lead to incomplete or misleading conclusions, making the combination of these products a crucial step in fire monitoring and analysis.